# Candidates of keyword

## Load model

In [ ]:
# import json
# import os
# # from training import create_PL
# from modules import get_pl_keywords
# import requests
# from google_map_API import get_location_range

## Load data

In [1]:
def get_all_raw_pl():
    
    import json
#     from training import create_PL
    from modules import get_pl_keywords
    import requests
    from google_map_API import get_all_location_range
     
    print("load data")
    total_data = [] 
    r = requests.get('https://steevebot.ml/all')  
    ori_data = r.json()
    key = list(ori_data.keys())
    to_DB = []
    all_loc = []
    for k in key: 
        data = []
    
        for num, job_num in enumerate(ori_data[k]):
            if num%500 ==0:
                print(num)
           
            temp=[]

            pl_des = get_pl_keywords(job_num["jobDescription"])
#                 print(pl_des)
            pl_ski = get_pl_keywords(job_num["skills"])
#                 print(pl_ski)
            data.append(pl_des+pl_ski)
            str_data = ''
            str_data = " ".join (pl_des+pl_ski) 
#                 print(str_data)
            
              
            # location
#             str_loc = ''
            all_loc.append(job_num['Location'])
#             location_range = get_location_range(job_num['Location'])
#             str_loc = " ".join(location_range)
            temp.append(job_num["JobID"])
            temp.append(k)
            temp.append(str_data)
            to_DB.append(temp)
            
            
        
        total_data.append(data)
    #### save PL and location to DB ####
    str_loc = get_all_location_range(all_loc)

    for n,l in enumerate(to_DB):
        s = ",".join(str_loc[n])
        l.append(s)
    create_PL(to_DB)
    
    return total_data

In [ ]:
# r = requests.get('https://steevebot.ml/all')
# ori_data = r.json()
# key = list(ori_data.keys())
print(type(ori_data['Backend'][0]['Location']))

In [ ]:
from google_map_API import get_location_range



In [ ]:
get_location_range('taichung')
get_location_range('Oklahoma City, OK')

In [ ]:
pwd

In [2]:
a,b = get_all_raw_pl()



load data
0
[['413078', 'Backend', 'android android kotlin redux SQLite mobile android mobile android Java kotlin rest mobile sql nosql React mobile kotlin redux Java sql', 'Chicago,Cook County'], ['413097', 'Backend', 'service web mobile this performance process process action performance linux system python Ruby docker kubernetes ansible deployment jvm rest sql protocols deployment git Jenkins linux python Ruby', 'Chicago,Cook County'], ['413887', 'Backend', 'web this web web web this javascript html css frameworks process web web React angular javascript css3 html5 web security xss javascript css javascript ajax dom jsonp web javascript React redux', 'Chicago,Cook County'], ['1804817', 'Backend', 'Java spring hibernate angularjs React interface module validation Java spring angular javascript junit cucumber protractor rabbitmq PostgreSQL Kibana logstash Jenkins docker this website http Java angular', 'Chicago,Cook County'], ['932884', 'Backend', 'system this deployment sharepoint de

ValueError: too many values to unpack (expected 2)

In [ ]:
for k in key:
    print(k)
print(key[0])

In [ ]:
def training_DNN():
    from DNN_model import get_Dnn_model
    total_data = get_all_raw_pl()
    
    # training DNN model
    get_Dnn_model(total_data)
    
def get_jobs(cv):
    from DNN_model import get_predict_field
    from modules import get_pl_keywords,top_k

    import TFIDF
    from google_map_API import location_filter
    
    try:
        pl_cnt = TFIDF.TFIDF.tfidf_dict
        
        print('pl_cnt exists')
    except:
        total_data = get_all_raw_pl()
        pl_cnt, word = TFIDF.TFIDF(total_data).get_tfidf()
        
    
    cv_PL = get_pl_keywords(cv)
    predict_field = get_predict_field(cv_PL,pl_cnt)
    
    #### get field data from DB ####
    p = get_field_PL(predict_field)
    posts_predict_field = []
    
    ### location filter
#    ,{'location':j.PL_location }
    for j in p:
        if location_filter(user_location,j.PL_location):
            posts_predict_field.append({'id':j.Job.job_ID},{'PL':j.PL})
    # posts_predict_field = GET DB DATA
    
    job_candidates = pick_k_jobs(cv_PL, posts_predict_field)
    
# def get_applicants():
    

In [ ]:
get_jobs('i like to write javascript and html. i am good at css. i want to be in java.')

In [ ]:
training_DNN()

In [ ]:
x = dict({'s':'12'})
print(x['s'])

In [ ]:
for i in data[0]:
    print(i)
# print(data[0])

In [ ]:
tf_idf = TFIDF(total_pl)
# from TFIDF import tf_idf
pl_cnt, words = tf_idf.get_tfidf()
print(pl_cnt)

In [ ]:
#### Return jobID, Field,PL[]